In [ ]:
'''What all do i need in the code:
For the simple structure:
1) Word embedding layer, feeed input layer
    1) Creating the gen batch function-> Seq and padded sequence
    
    
    a) Feed input layer: Seq length and padded sequence
    b) Labels: Padded too and >>>>>>CHUNKS<<<<<<
    c) Word embedding according to vocab loaded 
    
2)DONE  Trim vocab and word embeddings based on vocab in both domains. DONE

3) Run through RNN
    a) Ideally, have function here to append seq2seq and tree kernel representation
    b) Computing tree kernel and gaining a good represnation out of it
    
4) Decoding: 
    a) TRAINING:  <F1 measure< Allow for both with and without
    b) PREDICTIONS:
    1) F1 measure
    2) Allow for with and without crf
    3) Store model and allow for training continuation
    4) Predictions with crf code


'''

In [10]:
'''Create joint vocab'''
def get_min_vocab(domain_vocabs_list):
    min_vocab_size = len(domain_vocabs_list[0])
    for domain_vocab in domain_vocabs_list:
        if(len(domain_vocab)<min_vocab_size):
            smallest_vocab = domain_vocab
            min_vocab_size = len(smallest_vocab)
    return smallest_vocab

def get_common_words_across_domains(domain_vocabs_list):
    #Simple optimum -> Start with domain with smallest vocab, and then keep intersecting-> counting freq will be a costlier operation
    smallest_vocab = get_min_vocab(domain_vocabs_list)
    f_set = set(smallest_vocab.keys())
    for domain_vocab in domain_vocabs_list:
        set1 = set(domain_vocab.keys())
        f_set = f_set.intersection(set1)
    
    return f_set

def get_most_prominent_common_words(domain_names, domain_freqs_list, remove_stop_words = False):
    '''Input: MAKE SURE THAT domains_names is in order of domain_freqs_list
    Output: Dictionary of words ordered by counts in domains and cumulative freq
    To obtain format: Key ["<ORDER_OF_DOMAINS>"] 
    '''
    assert len(domain_names) == len(domain_freqs_list) #Just a small limited check
    common_word_dict = {}
    common_words = get_common_words_across_domains(domain_freqs_list)
    common_word_dict["<ORDER_OF_DOMAINS>"] = domain_names+["Cumulative_count"]
    for word in common_words:
        if(remove_stop_words and word in stop_words):
            continue
        counts = []
        for domain_freq_dict in domain_freqs_list:
            counts.append(domain_freq_dict[word])
        counts.append(reduce(lambda x,y:x+y, counts))
        common_word_dict[word] = counts
    
    return common_word_dict

def get_domain_vocab_and_freq(domain_names):
    vocab_freq_list = []
    for domain in domains:
        with open(domain_vfreq_dir.format(domain, domain)) as p1:
            vocab_freq_list.append(pickle.load(p1))
    return vocab_freq_list

'''Create joint vocab'''
def get_joint_vocab(domains, domain_general_path = "./Final_joint_data/Domains/{}/",output_path = "./Final_joint_data/"):
    output_pkl_vocab = output_path+'vocab_to_id.pkl'
    output_pkl_vocab_freq = output_path+'vocab_to_freq.pkl'
    tr_data_pkl = domain_general_path+"Normal__normal_training_list.pickle"
    
    vocab2id = {}
    vocab2freq = {}
    for domain in domains:
        with open(tr_data_pkl.format(domain),'r') as p1:
            tr_data = pickle.load(p1)
            for row in tr_data: #go through each row
                for word in row[0]:
                    if(word in vocab2id.keys()):
                        vocab2freq[word]+=1
                    else:
                        vocab2id[word] = len(vocab2id)
                        vocab2freq[word] = 1 
    vocab2id['<UNK>'] = len(vocab2id)
    vocab2id['<START>'] = len(vocab2id)
    vocab2id['<END>'] = len(vocab2id)
    with open(output_pkl_vocab,'w') as p1:
        pickle.dump(vocab2id,p1)
    with open(output_pkl_vocab_freq,'w') as p1:
        pickle.dump(vocab2freq,p1)
    
    return vocab2id, vocab2freq
    
vocab1, vocab2 = get_joint_vocab(["Laptop","Rest"])    

In [11]:
import numpy as np

In [50]:
from gensim.models import Word2Vec

In [53]:
x = Word2Vec.load('./Final_joint_data/Embeddings/Gensim_w2vec/laptop_200d')

In [54]:
x.wv.save_word2vec_format("./Final_joint_data/Embeddings/Gensim_w2vec/only_laptop_200d.txt")
del x

In [55]:
x.most_similar("food")

NameError: name 'x' is not defined

In [13]:
import numpy as np
import pickle
import sys
def trim_word_vectors(input_path, output_path, vocab_path, dim=200, use_glove_format = True):
    #Go through each line of embeddings
    #if in vocab then add 
    #Outputs vectors in npy format allowing for direct loading in model
    with open(vocab_path,'r') as p1:
        vocab = pickle.load(p1)
        
    embeddings = np.zeros([len(vocab), dim])
    
    print("Reading word vectors from file")
    with open(input_path) as f:
        if(not use_glove_format):
            next(f)
        for line in f:
            line = line.strip().split(' ')
            word = line[0]
            embedding = [float(x) for x in line[1:]]
            if(word in vocab):
                word_id = vocab[word]
                embeddings[word_id] = np.asarray(embedding)
    np.savez_compressed(output_path, embeddings = embeddings)
    print("Saved trimmed word vectors in np compressed format at: {}".format(output_path))
    
if(__name__ == "__main__"):
    word_embedding_path = sys.argv[1]
    output_path = sys.argv[2]
    vocab_path = sys.argv[3]
    n_dims = sys.argv[4]
    use_glove = bool(int(sys.argv[5]))
    trim_word_vectors(word_embedding_path, output_path, vocab_path, n_dims, use_glove)

IndexError: list index out of range

In [25]:
def trim_word_vectors(input_path, output_path, vocab_path, dim=200, use_glove_format = True):
    #Go through each line of embeddings
    #if in vocab then add 
    #Outputs vectors in npy format allowing for direct loading in model
    with open(vocab_path,'r') as p1:
        vocab = pickle.load(p1)
        
    embeddings = np.zeros([len(vocab), dim])
    
    print("Reading word vectors from file")
    with open(input_path) as f:
        if(not use_glove_format):
            next(f)
        for line in f:
            line = line.strip().split(' ')
            word = line[0]
            embedding = [float(x) for x in line[1:]]
            if(word in vocab):
                word_id = vocab[word]
                embeddings[word_id] = np.asarray(embedding)
    np.savez_compressed(output_path, embeddings = embeddings)
    print("Saved trimmed word vectors in np compressed format at: {}".format(output_path))

In [58]:
trim_word_vectors("./Final_joint_data/Embeddings/together_200d.txt", "np_Togetherw2vec_200d_trimmed","Final_joint_data/vocab_to_id.pkl", 200, False)

Reading word vectors from file
Saved compressed word vectors at: np_Togetherw2vec_200d_trimmed


In [7]:
tr_data_pkl = "./Final_joint_data/Domains/Rest/Normal__normal_training_list.pickle"
with open(tr_data_pkl, 'r') as p1:
    tr_data = pickle.load(p1)

In [8]:
tr_data[0]

(['but', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us', ' <PERIOD> '],
 [5, 5, 1, 5, 5, 3, 5, 5, 5])

In [9]:
len(tr_data)

3044

In [2]:
with open('./Final_joint_data/tag2id.pkl','r') as p1:
    x = pickle.load(p1)

In [3]:
x

{'<END>': -1, '<START>': 0, 'B-A': 1, 'B-O': 3, 'I-A': 2, 'I-O': 4, 'O': 5}

In [18]:
vocab1, vocab2 = get_joint_vocab(["Laptop","Rest"])

In [ ]:
def trimmed_vectors(input_word_vector_file, input_vocab):
    output_path = input_word_vector_file + "trimmed.txt"
    